In [1]:
from elasticsearch import Elasticsearch
import numpy as np
import pandas as pd
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
es.info().body

{'name': '0941564de984',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': '02rmGC-OT-mV4UBDyFr-sA',
 'version': {'number': '8.3.3',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '801fed82df74dbe537f89b71b098ccaff88d2c56',
  'build_date': '2022-07-23T19:30:09.227964828Z',
  'build_snapshot': False,
  'lucene_version': '9.2.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

In [2]:
df = pd.read_csv('kinopoisk4.csv', sep=";")
df.head()

,Unnamed: 0,link,eng_name,rus_name,year,duration,country,genre,director_name,rating
0,0,https://www.kinopoisk.ru/film/1249172/,NaN,Последний богатырь: Корень зла,2020.0,121.0,Россия,фэнтези,Дмитрий Дьяченко,NaN
1,1,https://www.kinopoisk.ru/film/8366/,The Thing,Нечто,1982.0,109.0,США,ужасы,Джон Карпентер,7.9
2,2,https://www.kinopoisk.ru/film/79850/,NaN,Ночной дозор,2004.0,115.0,Россия,фэнтези,Тимур Бекмамбетов,NaN
3,3,https://www.kinopoisk.ru/film/8151/,Interview with the Vampire: The Vampire Chroni...,Интервью с вампиром,1994.0,123.0,США,драма,Нил Джордан,8.0
4,4,https://www.kinopoisk.ru/film/4322004/,NaN,Плакать нельзя,2022.0,125.0,Россия,триллер,Наталья Назарова,NaN


In [3]:
df[['year', 'duration']] = df[['year', 'duration']].fillna(0)
df[['year', 'duration']] = df[['year', 'duration']].astype(int)
df[['link', 'eng_name', 'rus_name', 'country', 'genre', 'director_name']] = df[['link', 'eng_name', 'rus_name', 'country', 'genre', 'director_name']].fillna("nan")
df[['link', 'eng_name', 'rus_name', 'country', 'genre', 'director_name']] = df[['link', 'eng_name', 'rus_name', 'country', 'genre', 'director_name']].astype(str)
df[['rating']] = df[['rating']].fillna(-1.0)

In [7]:
es.options(ignore_status=[400,404]).indices.delete(index='movies')

ObjectApiResponse({'acknowledged': True})

In [8]:
mappings = {
        "properties": {
            "link": {"type": "keyword"},
            "eng_name": {"type": "text", "analyzer": "english"},
            "rus_name": {"type": "text", "analyzer": "russian", "search_analyzer": "custom_search_analyzer", "analyzer": "custom_search_analyzer"},
            "year": {"type": "integer"},
            "duration": {"type": "integer"},
            "country": {"type": "text", "analyzer": "russian"},
            "genre": {"type": "text", "analyzer": "russian"},
            "director_name": {"type": "text", "analyzer": "russian"},
            "rating": {"type": "float"}
    }
}

settings = {
    "analysis": {
      "filter": {
         "synonym_filter": {
            "type": "synonym",
            "synonyms": [
                "дары, подарки, Дары, Подарки"
            ]
         },
        "ru_stemmer": {
          "type": "stemmer",
          "language": "russian"
        }
      },
      "analyzer":{
         "custom_search_analyzer": {
            "tokenizer": "standard",
            "filter": [
               "lowercase",
               "synonym_filter",
               "ru_stemmer"
            ]
            }
      }
    }
  }

es.indices.create(index="movies", settings = settings, mappings=mappings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'movies'})

In [9]:
from elasticsearch.helpers import bulk

bulk_data = []
for i,row in df.iterrows():
    bulk_data.append(
        {
            "_index": "movies",
            "_id": i,
            "_source": {        
                "link": row["link"],
                "eng_name": row["eng_name"],
                "rus_name": row["rus_name"],
                "year": row["year"],
                "duration": row["duration"],
                "country": row["country"],
                "genre": row["genre"],
                "director_name": row["director_name"],
                "rating": row["rating"],
            }
        }
    )
bulk(es, bulk_data)

(1000, [])

In [10]:
es.indices.refresh(index="movies")
es.cat.count(index="movies", format="json")

ListApiResponse([{'epoch': '1670764308', 'timestamp': '13:11:48', 'count': '1000'}])

In [22]:
from IPython.display import display

# pip install ipymarkup
from ipymarkup import show_span_ascii_markup as show_markup

from yargy import (
    Parser,
    or_, rule,  and_
)
from yargy.pipelines import morph_pipeline, caseless_pipeline
from yargy.predicates import (
    eq, in_, dictionary, gte, lte, normalized,
    type, gram, is_capitalized, length_eq
)
from yargy.tokenizer import MorphTokenizer
from yargy import interpretation as interp
from yargy.interpretation import fact, attribute

from natasha import NamesExtractor


def show_matches(rule, *lines):
    parser = Parser(rule)
    for line in lines:
        matches = parser.findall(line)
        matches = sorted(matches, key=lambda _: _.span)
        spans = [_.span for _ in matches]
        show_markup(line, spans)
        if matches:
            facts = [_.fact for _ in matches]
            if len(facts) == 1:
                facts = facts[0]
            display(facts)

def get_match(rule, *lines):
    parser = Parser(rule)
    for line in lines:
        matches = parser.findall(line)
        matches = sorted(matches, key=lambda _: _.span)
        spans = [_.span for _ in matches]
        if matches:
            facts = [_.fact for _ in matches]
            if len(facts) == 1:
                facts = facts[0]
            return facts


INT = type('INT')
NOUN = gram('NOUN')
ADJF = gram('ADJF')
PRTF = gram('PRTF')
GENT = gram('gent')
DOT = eq('.')

TOKENIZER = MorphTokenizer()

In [106]:
Year = fact(
    'Year',
    ['year']
)

YEAR_VAL = and_(
    gte(1000),
    lte(2100)
)

YEAR_WORD = or_(
    rule('г', eq('.').optional()),
    rule(normalized('год'))
)

YEAR = rule(
    YEAR_VAL.interpretation(
        Year.year.custom(int)
    ),
    YEAR_WORD.optional(),
).interpretation(
    Year
)

Year_range = fact(
    'Year_range',
    ['start', 'stop']
)

YEAR_DELIMETOR_BEFORE = morph_pipeline([
    'с',
    'от',
    'новее'
])

YEAR_DELIMETOR_UNDER = morph_pipeline([
    '-',
    'по',
    'до'
])

LITERALS = {
    'тридцатые': 1930,
    'сороковые': 1940,
    'пятидесятые': 1950,
    'шестидесятые': 1960,
    'семидесятые': 1970,
    'восьмидесятые': 1980,
    'девяностые': 1990,
    'двухтысячные': 2000,
}
LITERAL = caseless_pipeline(LITERALS).interpretation(
    Year_range.stop.normalized().custom(LITERALS.get)
)

YEAR_RANGE = or_(
    rule(
        LITERAL
    ),
    rule(
        YEAR_DELIMETOR_BEFORE.optional(),
        YEAR.interpretation(
            Year_range.start
        ),
        YEAR_DELIMETOR_UNDER.optional(),
        YEAR.optional()
            .interpretation(
                Year_range.stop
        )
    )
    

).interpretation(
    Year_range
)


class Year_obj:
    start = 1000
    stop = 2100

    def __init__(self, years): 
        if years != None and years.start == None and years.stop != None:
            y = years.stop.year
            self.start = y
            self.stop = y + 9
        elif years != None and years.stop == None and years.start != None:
            self.start = years.start.year
            self.stop = 2100
        elif years != None:
            self.start = years.start.year
            self.stop = years.stop.year
        else:
            self.start = 1000
            self.stop = 2100


def create_year(string):
    years = get_match(YEAR_RANGE, string)
    return Year_obj(years)

Country = fact(
    'Country',
    ['name']
)

LITERALS_COUNTRY = {
    'российский': "Россия",
    'американский': "США",
    'английский': "Британия",
    'канадский': "Канада",
    'индийский': "Индия",
    'новозеладский': "Новая Зеландия",
    'ирландский': "Ирландия",
    'японский': "Япония",
}

LITERAL_COUNTRY = morph_pipeline(LITERALS_COUNTRY).interpretation(
    Country.name.normalized().custom(LITERALS_COUNTRY.get)
)

Genre = fact(
    'Genre',
    ['name']
)

names = [
    'драма',
    'ужасы',
    'боевик',
    'мультфильм',
    'комедия',
    'криминал',
    'аниме',
    'биография',
    'триллер',
    'ужасы',
    'история',
    'фэнтези',
    'фантастика',
    'мелодрама',
    'приключения',
    'детектив',
    'военный',
    'мюзикл',
]

GENRE = morph_pipeline(names).interpretation(
    Genre.name.normalized()
)

def q1(request_txt, start_y, stop_y, country, genre): 
    q1 = es.search(
        index="movies",
        body= {
            "query": {
                "bool": {
                    "should": {
                        "multi_match": {
                            "query": request_txt,
                            "fields": [
                                "eng_name",
                                "rus_name",
                                "director_name"
                            ],
                            "type" : "cross_fields",
                            "analyzer" : "russian",
                            "prefix_length" : "2"
                        }
                    },
                    "filter": [{
                        "range": { "year": { "gte": start_y, "lte" : stop_y }}
                        },
                        { "query_string":{  
                        "query": f"genre:{genre} AND country:{country}"
                        } },
                    ]
                    
                }
            }
        }
    )
    display(q1['hits'])

def request1(input_str):
    year = create_year(input_str)
   
    country = get_match(LITERAL_COUNTRY, input_str)
    print(country)
    if country == None:
        country = '*'

    
    genre = get_match(GENRE, input_str)
    if genre == None:
        genre = '*'

    q1(input_str, year.start, year.stop, country, genre)


input_str = input()
request1(input_str)


Япония


C:\Users\danil\AppData\Local\Temp\ipykernel_8372\417387360.py:153: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  q1 = es.search(


{'total': {'value': 2, 'relation': 'eq'},
 'max_score': 0.0,
 'hits': [{'_index': 'movies',
   '_id': '346',
   '_score': 0.0,
   '_source': {'link': 'https://www.kinopoisk.ru/film/4766095/',
    'eng_name': 'One Piece Film: Red',
    'rus_name': 'Ван-Пис: Красный',
    'year': 2022,
    'duration': 115,
    'country': 'Япония',
    'genre': 'аниме',
    'director_name': 'Горо Танигути',
    'rating': -1.0}},
  {'_index': 'movies',
   '_id': '695',
   '_score': 0.0,
   '_source': {'link': 'https://www.kinopoisk.ru/film/4504336/',
    'eng_name': 'Gekijouban Jujutsu Kaisen 0',
    'rus_name': 'Магическая битва 0. Фильм',
    'year': 2021,
    'duration': 105,
    'country': 'Япония',
    'genre': 'аниме',
    'director_name': 'Пак Сон-ху',
    'rating': 7.7}}]}